# Automacao de navegação web com Selenium

Informações iniciais, do objetivo do projeto e bibliotecas utilizadas.

Sabia que é possível fazer uma automação completa na web, acessando sites, extraindo valores de páginas com a linguagem Python? É possível sim, e o nome do grande responsável é a biblioteca **Selenium.**

A biblioteca [Selenium](https://www.selenium.dev/) é abrangente quando lidados com automação de navegadores web. Fornece ferramentas para realizar a interação do usuário com navegador, a partir de um núcleo chamado [WebDriver](https://www.selenium.dev/documentation/webdriver/), uma simulação da interface onde conduz o navegador da mesma maneira como o usuário faria.

Nosso objetivo nesse projeto é verificar a cotação do *dólar, euro e ouro*, para atualizar uma lista de produtos que variam seu preço de acordo com a cotação da moeda. Para isso, vamos navegar com o webdriver pesquisando sobre cada cotação, extraindo seu valor, para atualizar nossa base de dados, que esta em uma planilha *Excel*.

Para iniciar, vamos realizar a instalação, se necessário, da biblioteca. O webdriver deve ser realizado o download de acordo com o navegador e sua versão. Para navegadores Chrome, o webdriver se chama **chromedriver**. Já para o Firefox, o webdriver chama **geckodriver**. A partir daí estamos aptos a importar as bibliotecas necessárias e começar a codificar.

A biblioteca utilizada foi a selenium, mas importamos os módulos *webdriver, Keys e By*, da biblioteca, para realização da execução. 

* `webdriver` é responsável por instanciar o navegador para acontecer a navegação;
* `Keys` será usada para simular a execução de importantes teclas;
* `By` será responsável por encontrar os elementos da linguagem web para acessá-los.

In [101]:
# apenas uma unica vez 
# !pip install selenium -q

In [102]:
# chrome -> chromedriver
# firefox -> geckodriver

In [103]:
from selenium import webdriver # controlar o navegador
from selenium.webdriver.common.keys import Keys # usar o teclado durante a navegação
from selenium.webdriver.common.by import By # localizar itens no navegador

## Cotação das moedas

Nessa etapa vamos fazer toda automação, fazendo com que seja pesquisa e extraída cada moeda que desejamos e o valor seja salva em uma variável.

Nosso objetivo é descobrir a cotação do dólar, euro e ouro para que, posteriormente, possamos atualizar nossa base de dados, e assim atualizar o preço final de cada produto.

Para isso, vamos utilizar o webdriver, entrar no site *Google*, através do método **get**, para depois pesquisar por cada elemento HTML da página. Vamos pesquisar os elementos pelo XPATH de cada, onde será possível resgatar os elementos que estamos interessados para realizar a pesquisa e extrair a cotação.

In [104]:
navegador = webdriver.Chrome() # instanciando o chrome na variavel navegador e abrindo a navegação
navegador.maximize_window() # maximizando a janela
navegador.get('https://www.google.com') # acessando o google

# - DOLAR -
# pesquisando cotação do dolar
navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação dolar')

# apertando enter
navegador.find_element(By.XPATH, 
                       '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER) # apertando enter

# extraindo o valor da cotação
cotacao_dolar = navegador.find_element(By.XPATH,
                       '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")
# - EURO -
navegador.get('https://www.google.com')
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação euro')
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
cotacao_euro = navegador.find_element(By.XPATH, '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute('data-value')


# - OURO - (nao aparece automaticamente no google então vamos acessar um site e extrair o valor)
navegador.get('https://www.google.com')
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys('cotação ouro')
navegador.find_element(By.XPATH, '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys(Keys.ENTER)
navegador.get('https://www.melhorcambio.com/ouro-hoje')
ouro = navegador.find_element(By.XPATH, '//*[@id="comercial"]').get_attribute('value')
cotacao_ouro = ouro.replace(',', '.')

# fechando navegador
navegador.quit()

#print('Dolar: {}'.format(cotacao_dolar))
#print('Euro: {}'.format(cotacao_euro))
#print('Ouro: {}'.format(cotacao_ouro))

## Atualizando base de dados

Realizado toda automação para extrair a cotação de cada moeda que estamos interessados, vamos partir para atualizar nossa base de dados. Primeiro, vamos acessá-la para depois atualizar o preço que compra e venda, ambos atrelados a cotação de moeda, tais como dólar, euro e ouro.

### Acessando dados

O primeiro passo é acessar o arquivo de dados e, assim que acessado, vamos realizar a atualização das cotações das moedas com os valores antigos, para a cotação que foi extraída a partir do processo de automação.

In [105]:
import pandas as pd
import time

dados = pd.read_excel('/home/particular/Downloads/GUIRRA DATATEC/LIRA HASTAG/Aula 3/Aula 3/Produtos.xlsx')

display(dados)
time.sleep(5)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


In [106]:
# atualizar dados da cotação - dolar
dados.loc[dados['Moeda'] == 'Dólar', 'Cotação'] = float(cotacao_dolar)
dados.loc[dados['Moeda'] == 'Euro', 'Cotação'] = float(cotacao_euro)
dados.loc[dados['Moeda'] == 'Ouro', 'Cotação'] = float(cotacao_ouro)

#display(dados)

### Atualizando o preço de Compra e de Venda

Como cada moeda teve seu valor atualizado, podemos então calcular o preço de compra e venda de cada produto. Como cada produto é atrelado a uma moeda especifíca, vamos realizar a atualização seguindo a informação do valor da moeda. O preço de compra é dado por,

$$\mathrm{Preço \; de \; Compra} = \mathrm{Preço \; Original} * \mathrm{Cotação}.$$

Já o Preço de Venda é calculado em cima do preço de Compra, dado por, 

$$\mathrm{Preço \; de \; Venda} = \mathrm{Preço \; de \; Compra} * \mathrm{Margem}.$$

Com isso, obtemos o valor final de cada produto para realização das vendas.

In [107]:
dados['Preço de Compra'] = dados['Preço Original'] * dados['Cotação']
dados['Preço de Venda'] = dados['Preço de Compra'] * dados['Margem']
display(dados)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.011300,5011.249887,1.40,7015.749842
1,Carro Renault,4500.00,Euro,5.508747,24789.360123,2.00,49578.720246
2,Notebook Dell,899.99,Dólar,5.011300,4510.119887,1.70,7667.203808
3,IPhone,799.00,Dólar,5.011300,4004.028700,1.70,6806.848790
4,Carro Fiat,3000.00,Euro,5.508747,16526.240082,1.90,31399.856156
5,Celular Xiaomi,480.48,Dólar,5.011300,2407.829424,2.00,4815.658848
6,Joia 20g,20.00,Ouro,324.630000,6492.600000,1.15,7466.490000


### Exportando arquivo de dados

Como todo processo de automação foi concluído, podemos exportar o arquivo e visualiza - lo no excel.

In [108]:
dados.to_excel('Produtos-info.xlsx')

Espero que tenha gostado dessa simples demonstração de automação com python através da biblioteca selenium. As aplicações são vastas e partir do básico, podemos ir evoluindo e criando diferentes aplicações, no intuito de facilitar nosso trabalho no dia a dia.